In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [77]:
X_train = pd.read_csv("train.csv")
y_train = pd.read_csv("train_label.csv", header=None)# Data no incluye primer valor como columna

In [78]:
X_test = pd.read_csv("test.csv")
y_test = pd.read_csv("test_label.csv", header=None) #Data no incluye el primer valor como columna

In [82]:
display(X_train.head(), X_train.shape, y_train.shape)

,Unnamed: 0,v0,v1,v2,v3,v4,v5,v6,v7,v8,...,ae61,ae62,ae63,ae64,ae65,ae66,ae67,ae68,d69,d70
0,1651,0.00,143270.10,100.0,0.0,0.0,63090564.44,4199850.00,0.0,6240000.0,...,0,0,0,0,0,0,0,0,0,0
1,1370,0.00,1403461.97,90.0,10.0,29.0,8013913.78,8177030.69,44951.9,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2649,19800.24,63401.41,100.0,0.0,0.0,323384.60,67610.22,0.0,0.0,...,1,0,0,0,0,0,0,0,1,0
3,564,0.00,10585032.22,100.0,0.0,5.0,29596681.95,186964.54,0.0,4500000.0,...,0,0,0,0,0,0,0,0,0,0
4,2728,0.00,11259663.09,78.0,22.0,0.0,33626595.71,0.00,0.0,3972827.7,...,0,1,0,0,0,0,0,0,0,0


(2189, 72)

(2189, 1)

Entiendo que los valores ya fueron convertidos a OneHotEnconder, y que cada una de las letras representan una variable y el número que lo acompaña representa la categoria de dicha varible

In [79]:
y_train.columns = ["output"]
y_test.columns = ["output"]

In [40]:
#Corroboramos el porcentaje de imbalance data dentro del dataaset
imbalance = y_train.output.value_counts()
imbalance_result = [x/imbalance.sum()*100 for x in imbalance]
pd.DataFrame(imbalance_result, index=["Pagado a Tiempo", "No Pagado a Tiempo"], columns=["Proporcion (%)"])

,Proporcion (%)
Pagado a Tiempo,80.393053
No Pagado a Tiempo,19.606947


In [83]:
#Corroboramos el porcentaje de imbalance data dentro del dataset de testing 
imbalance_test = y_test.output.value_counts()
imbalance_result_test = [x/imbalance.sum()*100 for x in imbalance]
pd.DataFrame(imbalance_result_test, index=["Pagado a Tiempo", "No Pagado a Tiempo"], columns=["Proporcion (%)"])

,Proporcion (%)
Pagado a Tiempo,80.393053
No Pagado a Tiempo,19.606947


### Consideracion para imbalanced data.

Ambos modelos (training and testing) estan distribuidos en una proporcion de 80:20, por lo cual debemos considerar que al momento de evaluar accuracy debe ser mayor al 80% (si de realizar el modelo en base al data imbalanced), ya que en el peor de los escenarios el modelo predecira el 80% de las veces bien el testing.

In [60]:
display(X_train.describe(), X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2189 entries, 0 to 2188
Data columns (total 72 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2189 non-null   int64  
 1   v0          2189 non-null   float64
 2   v1          2189 non-null   float64
 3   v2          2189 non-null   float64
 4   v3          2189 non-null   float64
 5   v4          2189 non-null   float64
 6   v5          2189 non-null   float64
 7   v6          2189 non-null   float64
 8   v7          2189 non-null   float64
 9   v8          2189 non-null   float64
 10  v9          2189 non-null   float64
 11  v10         2189 non-null   float64
 12  v11         2189 non-null   float64
 13  v12         2189 non-null   float64
 14  v13         2189 non-null   float64
 15  v14         2189 non-null   float64
 16  v15         2189 non-null   float64
 17  v16         2189 non-null   float64
 18  v17         2189 non-null   float64
 19  v18         2189 non-null  

,Unnamed: 0,v0,v1,v2,v3,v4,v5,v6,v7,v8,...,ae61,ae62,ae63,ae64,ae65,ae66,ae67,ae68,d69,d70
count,2189.000000,2.189000e+03,2.189000e+03,2189.000000,2189.000000,2189.000000,2.189000e+03,2.189000e+03,2.189000e+03,2.189000e+03,...,2189.000000,2189.000000,2189.000000,2189.000000,2189.000000,2189.000000,2189.000000,2189.000000,2189.000000,2189.0
mean,1457.217908,2.789336e+04,2.498074e+06,87.098675,10.317953,3.608040,7.332287e+07,1.011043e+07,1.126268e+04,2.283687e+06,...,0.034719,0.371859,0.004111,0.021928,0.004568,0.014619,0.007766,0.044312,0.122430,0.0
std,842.158691,2.770920e+05,3.713769e+06,29.548930,25.922405,8.477891,3.009257e+08,4.209394e+07,1.540442e+05,3.283374e+06,...,0.183109,0.483412,0.064003,0.146481,0.067450,0.120048,0.087803,0.205835,0.327857,0.0
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,731.000000,0.000000e+00,1.411988e+04,100.000000,0.000000,0.000000,2.898817e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,1453.000000,0.000000e+00,1.781006e+05,100.000000,0.000000,0.000000,1.623024e+07,1.970460e+04,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,2188.000000,0.000000e+00,3.958601e+06,100.000000,0.000000,5.000000,3.308427e+07,2.560791e+06,0.000000e+00,4.355600e+06,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,2907.000000,5.030246e+06,1.257078e+07,100.000000,100.000000,183.000000,3.283312e+09,4.415314e+08,2.427617e+06,1.000000e+07,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


None

Consideramos que los primeros 38 columnas (aquellas que tienen como prefijo 'v'),  seran considerados vairables continuas, aunque probablemente la primera columna (Unnamed: 0) sea algun id del dataset, el resto seran datos resultados de categorias.

Para futuras consideraciones sobre un pipeline:
- Para modelos lineales:
* A estos primeros debemos estandarizarlos (v0-v37). Utilizando column_transformer a las varibles con prefijo 'v' 
* Resampling: Oversampling o undersampling (evaluaremos dependiendo del performance del modelo bajo ambos escenarios y elegiremos el que mejor se adecue)
* Utilizare la libreria eli5 para encontrar el "weight" que aporta al modelo en base a LogisticRegression o SVC
* Aplicar RandomizedSearchCV para calibrar los mejores parametros y evaluar mediante confusion_matrix, Classification_report y roc_auc (para evaluar en funcion de gini)
 
Note: Utilizare primero el modelo lineal para tener una referencia por el cual guiarme para hacer los modelos no lineales

-Para otros modelos:
* Utilizar los mismos atributos del modelo para evaluar el feature_importance (particularmente ExtraTree)
* Aplicar RandomizedSearchCV y realizar las evaluaciones metricas.


In [44]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

## Modelo Lineal

In [47]:
 from sklearn.preprocessing import StandardScaler
 from sklearn.compose import ColumnTransformer

In [65]:
#Datos a ser utilizados para la standarization
col_standard = X_train.iloc[:,0:38].columns.tolist()
print(col_standard)

['Unnamed: 0', 'v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36']


In [71]:
ct = ColumnTransformer([("stand", StandardScaler(), col_standard)], remainder='passthrough')
X_train_stand = pd.DataFrame(ct.fit_transform(X_train), columns=X_train.columns)
X_train_stand.head()

,Unnamed: 0,v0,v1,v2,v3,v4,v5,v6,v7,v8,...,ae61,ae62,ae63,ae64,ae65,ae66,ae67,ae68,d69,d70
0,0.230154,-0.100688,-0.634219,0.436709,-0.398123,-0.425680,-0.034011,-0.140446,-0.073130,1.205229,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.103588,-0.100688,-0.294812,0.098210,-0.012268,2.995764,-0.217076,-0.045941,0.218748,-0.695690,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.415475,-0.029214,-0.655730,0.436709,-0.398123,-0.425680,-0.242639,-0.238636,-0.073130,-0.695690,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1.060871,-0.100688,2.178058,0.436709,-0.398123,0.164225,-0.145339,-0.235800,-0.073130,0.675165,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.509303,-0.100688,2.359757,-0.307989,0.450757,-0.425680,-0.131944,-0.240242,-0.073130,0.514570,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
lr = LogisticRegression()

### Proceso de Oversampling con SMOTE

In [80]:
sm =SMOTE(sampling_strategy='minority',  random_state=6)
X_train_sm, y_train_sm = sm.fit_resample(X_train_stand, y_train)
y_train_sm['output'].value_counts()

1.0    1760
0.0    1760
Name: output, dtype: int64